In [ ]:
# HSPd

## Last.fm Dataset

In [ ]:
import glob
from collections import defaultdict
import os.path

import pandas as pd

lfm_files = glob.glob('../../data/hit_song_prediction_lastfm/sample/*.pickle')


def merge_features(cols):
    cols = map(lambda c: '.'.join(str(c).split('.')[:2]), cols)
    cols = sorted(list(set(cols)))
    
    return cols
    
ess_lfm_features = None
for f in lfm_files:
    columns = list(filter(lambda c: not c.startswith('metadata') ,pd.read_pickle(f).columns))
    columns = merge_features(columns)
    
    if 'melspect' not in f:
        if ess_lfm_features:
            ess_lfm_features.intersection_update(set(columns))
        else:
            ess_lfm_features = set(columns)

ess_f = sorted(list(filter(lambda f: (f.startswith('highlevel') or f.startswith('lowlevel') or f.startswith('rhythm') or f.startswith('tonal')), ess_lfm_features)))

feat = defaultdict(list)
for f in ess_f:
    f = f.split('.')
    feat[f[0]].append(f[1])
    
display(dict(feat))

In [ ]:
import glob
import os.path

import pandas as pd

cleaned_files = glob.glob('../../data/hit_song_prediction_ismir2020/sample/*.pickle')
cleaned_files = list(filter(lambda f: 'cleaned' in f, cleaned_files))
    
ess_cleaned_features = None
for f in cleaned_files:
    columns = list(filter(lambda c: not str(c).startswith('metadata') ,pd.read_pickle(f).columns))
    columns = merge_features(columns)
    
    if 'melspect' not in f:
        if ess_cleaned_features:
            ess_cleaned_features.intersection_update(set(columns))
        else:
            ess_cleaned_features = set(columns)
            
display(ess_cleaned_features - ess_lfm_features)